In [ ]:
import pandas as pd
train = pd.read_csv('../Data/reviews_training_26000.csv')
test = pd.read_csv('../Data/reviews_test_4000.csv')
train = train[['review','sentiment']]
test = test[['review','sentiment']]

In [ ]:
# import nltk
# nltk.download('stopwords') # For Stop word removal
import re
import string 
def pre_process(text):
    text = text.lower()
    text = re.sub(r'<br\s*/?>', ' ', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    #Space Removal
    text = re.sub("^\s+|\s+$", "", text)
    text = re.sub("\s+", " ", text)
    #Removing Numbers
    text = ''.join([i for i in text if not i.isdigit()])
    #Stop Word Removal 
    # tokens = nltk.word_tokenize(text)
    # stop_words = nltk.corpus.stopwords.words("english")
    # tokens = [token for token in tokens if token not in stop_words]
    #Stemming
    # stemmer = nltk.stem.PorterStemmer()
    # tokens = [stemmer.stem(token) for token in tokens]
    # pre_processed_text = " ".join(tokens)
    # return(pre_processed_text)
    return(text)
    
train['review'] = train['review'].apply(pre_process)
test['review'] = test['review'].apply(pre_process)

In [ ]:
train['sentiment'] = train['sentiment'].map({'positive':1,'negative':0})
test['sentiment'] = test['sentiment'].map({'positive':1,'negative':0})

In [ ]:
token_train = [str(i) for i in train['review'].values]
token_test = [str(i) for i in test['review'].values]

In [ ]:
from transformers import AutoTokenizer
# Roberta Tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
encoded_train = tokenizer.batch_encode_plus(token_train, truncation=True, padding=True, return_tensors='pt')

In [ ]:
encoded_test = tokenizer.batch_encode_plus(token_test, truncation=True, padding=True, return_tensors='pt')

In [ ]:
encoded_test['input_ids'].shape
encoded_train['input_ids'].shape

In [ ]:
import torch
input_ids = torch.tensor(encoded_train['input_ids'])
attention_masks = torch.tensor(encoded_train['attention_mask'])

In [ ]:
from transformers import RobertaModel
# Model Import
model = RobertaModel.from_pretrained('roberta-base')
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    logits = outputs[0]
    probs = logits.softmax(dim=1)

In [ ]:
inputs = tokenizer(token_val, padding=True, truncation=True, return_tensors='pt')

In [ ]:
import torch
# Use the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}
model.to(device)

# Model Predictions 
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs[0]
    probs = logits.softmax(dim=1)